In [1]:
# https://stackoverflow.com/questions/36798782/scipy-ckdtree-nearest-neighbor-including-zeros-distance
from scipy.spatial import cKDTree
import numpy as np
import pandas as pd

#load and normalize features
tracks = pd.read_csv('data/tracks.csv', index_col=0, header=[0, 1]);
#load and normalize features
features= pd.read_csv('data/features.csv', index_col=0, header=[0, 1, 2])
#dfmax=abs(features).max()
#features=features.divide(dfmax)

genres=pd.read_csv('data/genres.csv', index_col=0, header=[0])

In [2]:
# Keep only the genre for each track
label=pd.DataFrame({('label','genre','main'): tracks['track']['genre_top'],
                    ('label','genre','sub'): tracks['track']['genres']})

# Keep on the tracks from small subset
label = label[tracks['set','subset'] == 'small']


# Keep only the genres we want
genre_list = ['Rock','Folk','Hip-Hop']#['Hip-Hop', 'Folk',Rock','Jazz','Classical','Pop','Electronic']
label = label[label['label']['genre']['main'].isin(genre_list)]

#add sub genres
genres_dict=genres['title'].to_dict()
genres_dict={str(key):genres_dict[key] for key in genres_dict}

label['label']['genre']['sub']=label['label']['genre']['sub'].str.replace('[^0-9| ]','').str.split(' ').apply(lambda x:'|'.join([genres_dict[k] for k in x ]))


# Merging
dataSet = features.merge(label, left_index=True, right_on='track_id')
dataSet

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
140           0.533579 -0.623885 -1.086205 -1.081079 -0.765151 -0.072282   
141           0.172898 -0.284804 -1.169662 -1.062855 -0.706868 -0.708281   
182           0.300352  1.075603 -0.464930  0.145206 -0.310866  1.141097   
190          -0.326067 -0.849603 -0.829708 -0.807827 -1.176002 -0.662898   
193          -0.308888 -1.054326 -0.102362 -0.638125 -0.811252 -0.612962   
194          -0.803457 -0.146634 -1.123406 -1.040201 -1.234728 -0.999059   
197          -1.064454 -1.141880 -0.116212 -1.122205 -0.797917 -0.810862   
200          -1.218976  0.865863 -1.406675  2.389059 -1.551505 -0.578557   
203          -0.765698 -1.201390 -0.748472 -0.644162 -0.832776 -0.952697   
204          -0.952763 -0.128642 -0.424757 -0.475148 -0.432873 -0.240849   
207          -1.176386 -0.744855 -0.237885 -1.208433 -1.239303 -0.834120   
210          -1.139867 -0.765117 -0.398347 -0.873563  0.298781 -0.531524   
211          -0.340395 -0.202638 -0.216030 -0.078217  0.550447 -0.636939   
212           1.425728 -0.730105 -0.893947 -0.900159  1.302656 -0.584388   
255           0.114022  1.043898  0.171339  0.119401  0.344436  0.085586   
256           0.045324  5.285462 -0.347695 -0.216278 -0.554277 -0.322547   
368           1.227293  1.407863  0.113506  3.899308  4.060197  3.048976   
459           1.551306 -1.021888 -0.378807 -0.620683 -0.928914 -0.913499   
534          -0.363953 -0.734606  0.435940  0.085379  1.628014 -0.296006   
540           0.294533 -0.171833  0.294257  0.172662  0.694958  0.706916   
546           0.149045  0.415505 -0.076961 -0.215612  0.669312  0.491404   
574           0.242732  0.725443  1.167472 -0.392548  3.399552  0.213584   
602          -0.490929 -0.681323 -1.226509 -0.977671 -0.876011 -0.452854   
620           0.253041 -0.137290 -1.039891  0.097182 -0.837418 -0.703705   
621          -0.892292 -0.918542  1.260457  0.388367  0.445968  0.187796   
625           0.065721  0.136715  0.304754 -0.685980 -1.150357 -1.108921   
676          -0.092578 -1.211476 -0.470576 -0.456790 -0.914000  0.117604   
690          -0.024725  0.843350  1.829660  1.783442  2.232701  0.481475   
...                ...       ...       ...       ...       ...       ...   
149370        0.310121  0.476840 -1.172890  1.009710  0.531995 -1.154552   
149416        0.771231 -1.426435 -1.215955 -1.174301  0.536197 -0.047070   
149417        0.611416 -0.869314 -0.154737 -0.730425  0.827616  2.616121   
149452        0.663534 -0.725104 -0.244992  1.123667 -0.205687 -0.064139   
149488       -0.772412  0.275008  1.203154  0.294156 -1.172800 -1.054118   
149523       -0.283016 -0.479126  0.505299 -1.290345 -0.701493 -0.987702   
149623       -0.244286  0.452232 -0.218394 -0.589496 -0.161543  0.023167   
149625       -0.493274 -0.176755 -0.064061  0.223743  0.223735  0.984642   
149626        0.139201 -0.289042  0.347036  0.138737  0.233770 -0.043236   
149687        0.350609 -0.670352 -0.374351 -0.441257 -0.885404 -0.392050   
149689       -0.285116 -0.537384  0.360416  0.883256  1.040882  0.528060   
149690        1.351361  1.344398  0.696827  0.802784  0.127069  0.340849   
149951        1.307373  0.962894 -0.343956  0.732331  0.243892  0.100974   
149953       -0.083667  1.248844  1.808267  1.547573  2.501516  1.506756   
150062       -0.831557 -0.812309 -0.774834  1.092289 -0.341895  0.086679   
150063       -0.873661 -0.685591 -0.797073 -0.401801 -0.438679 -0.442689   
150064       -0.776794 -0.723828 -0.527783 -0.854116 -0.687187 -1.096633   
150287       -0.526006  0.188689  0.450782  0.62

In [3]:
# Convert to np.array
points_ref = np.array([tuple(x) for x in dataSet.drop(columns='label',level=0).values])
points_ref

array([[ 7.18065262e+00,  5.23030901e+00,  2.49320805e-01, ...,
         0.00000000e+00,  2.08987212e+00,  6.14481084e-02],
       [ 5.27562976e-01, -7.76543170e-02, -2.79610306e-01, ...,
         0.00000000e+00,  2.19330311e+00,  4.48606014e-02],
       [ 5.33579469e-01, -6.23884618e-01, -1.08620489e+00, ...,
         1.95312500e-03,  3.14396811e+00,  5.77124506e-02],
       ...,
       [-6.77708983e-01, -8.30608130e-01, -6.86122000e-01, ...,
         1.95312500e-03,  6.92838526e+00,  8.42977017e-02],
       [-5.85059345e-01, -6.35074914e-01, -1.94741905e-01, ...,
         8.78906250e-03,  3.44125104e+00,  1.29479930e-01],
       [ 4.42161486e-02, -3.00441355e-01, -2.17022225e-01, ...,
         0.00000000e+00,  9.68863487e+00,  3.07869967e-02]])

In [4]:
# Construct the KD Tree -> https://en.wikipedia.org/wiki/K-d_tree
tree = cKDTree(points_ref)

In [5]:
# Loading custom songs features
features_c = pd.read_csv('data/custom_features.csv', index_col=0, header=[0, 1, 2], sep=',', encoding='utf-8')
tid_c = pd.read_csv('data/custom_tid.csv', index_col=0, header=[0], sep=',', encoding='utf-8')

#dfmax=abs(features_c).max()
#features_c=features_c.divide(dfmax) TODO IMPORTANT GLAAAA
features_c

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
-1            1.851763  0.453497  1.255231  0.062612  1.363372 -0.167979   
-2            0.200710 -0.092617 -0.188147  0.380223  0.191017 -0.324527   
-3           -0.239222 -0.832653 -0.781708 -0.523184 -0.418843 -0.596029   
-4            1.556971 -0.406326  0.419612  0.238071 -0.333854  1.517641   

feature                                               ...      tonnetz  \
statistics                                            ...          std   
number            07        08        09        10    ...           04   
track_id                                              ...                
-1          0.630384  1.059093  0.234977  1.672356    ...     0.062819   
-2         -0.530401 -0.567166 -0.517418 -0.117201    ...     0.082100   
-3         -0.531793 -0.512148 -0.179240 -0.519247    ...     0.141308   
-4          0.799368  0.457635 -0.261897 -0.963094    ...     0.104300   

feature                               zcr                                \
statistics                       kurtosis       max      mean    median   
number            05        06         01        01        01        01   
track_id                                                                  
-1          0.015719  0.017776  22.354948  0.434082  0.053094  0.049805   
-2          0.016624  0.026085   4.095423  0.364746  0.056571  0.042480   
-3          0.021789  0.021882   7.581665  0.584473  0.077602  0.075684   
-4          0.020775  0.016373  11.180266  0.538086  0.070393  0.068848   

feature                                   
statistics       min      skew       std  
number            01        01        01  
track_id                                  
-1          0.000000  2.831216  0.030197  
-2          0.003418  1.840665  0.041089  
-3          0.000000  1.318142  0.035792  
-4          0.000000  1.373266  0.027817  

[4 rows x 518 columns]

In [6]:
# Convert to np.array
features_c_t = np.array([tuple(x) for x in features_c.values])

In [7]:
# https://stackoverflow.com/questions/36798782/scipy-ckdtree-nearest-neighbor-including-zeros-distance
_, idx = tree.query(features_c_t, k=1)
dataSet.iloc[idx,:]

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
108967       -0.466435 -0.459286  0.083062  0.398263  0.131551  0.459454   
70208        -0.083487 -0.443800  0.327928  0.225605  0.051387  0.169189   
84743         2.080668  2.525734  3.562139  1.423534  0.324176  0.377587   
98582        -1.126292 -0.927522 -0.823718 -0.278688  1.101609 -0.167875   

feature                                                      ...            \
statistics                                                   ...             
number            07        08        09        10           ...             
track_id                                                     ...             
108967      0.205650  0.348076 -0.003418 -0.334279           ...             
70208       0.549522  0.658227 -0.093566 -0.562233           ...             
84743       0.386881  1.833050  0.918043  0.644126           ...             
98582       0.053799 -0.608424 -1.103718 -0.345277           ...             

feature      tonnetz        zcr                                          \
statistics       std   kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
108967      0.019326   1.448704  0.188965  0.064803  0.062988  0.000000   
70208       0.023032   3.978354  0.361328  0.050257  0.032715  0.001465   
84743       0.015954  85.071587  0.728027  0.071666  0.071777  0.000000   
98582       0.020203  67.102074  0.759766  0.053293  0.040527  0.000000   

feature                           label                          
statistics      skew       std    genre                          
number            01        01     main                     sub  
track_id                                                         
108967      0.701943  0.023736     Rock  Rock|Electro-Punk|Goth  
70208       1.881357  0.048649  Hip-Hop             Hip-Hop|Rap  
84743       6.292879  0.041527     Rock    Punk|Metal|Loud-Rock  
98582       7.748343  0.072091  Hip-Hop             Hip-Hop|Rap  

[4 rows x 520 columns]